# Exploratory data analysis on all the datasets

Main Points:
- how many nights for each recording?
- Is there any non-wear time?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import glob

mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
from nonwear import nimbaldetach

## Hom3ostasis
Notes: subject 2_SL has more than 2 days of recording

In [91]:
HOM_path = "/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Serena_Marcello/SleepAnalysis/Datasets/Hom3ostasis/"
HOM_path = "/Volumes/Untitled/Hom3_CinC/CinC/Data_24hPulses/mod_newQuality"
sub_names = sorted([f for f in os.listdir(HOM_path) if not f.startswith('.')])
len(sub_names)

39

In [ ]:
# Create an input_GGIR and an output_GGIR folder for each subject
for i, sub_ID in enumerate(sub_names): 
    input_GGIR = HOM_path + sub_ID + "/input_GGIR/"
    output_GGIR = HOM_path + sub_ID + "/output_GGIR/"
    if not os.path.exists(input_GGIR):
        os.makedirs(input_GGIR)
    if not os.path.exists(output_GGIR):
        os.makedirs(output_GGIR)
    print(f"{i+1}/{len(sub_names)} - {sub_ID} done!")

### Save acc data for GGIR and detect non-wear

In [14]:
fs_acc = 32
sub_names = ["2_SL"]
for i, sub_ID in enumerate(sub_names):
    print(sub_ID)
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    temp_path = glob.glob(HOM_path + sub_ID + '/**/TEMP.csv', recursive=True)[0]

    acc_data = pd.read_csv(acc_path, header = None)
    acc_start = acc_data.iloc[0,0] # start time in unix
    acc = acc_data.drop([0,1]).reset_index(drop=True) / 64 # convert to g
    acc.index = pd.date_range(start=pd.to_datetime(acc_start, unit='s'), periods = len(acc), freq = '0.03125s') # timestamps as index
    acc.columns = ['x', 'y', 'z']
    acc = acc.loc[acc.index[-1] - pd.Timedelta(hours = 16):] # keep only the last 15 hours
    # print recording duration
    print(f"Duration: {acc.index[-1] - acc.index[0]}")
    # Save for GGIR
    acc.to_csv(HOM_path + sub_ID + '/input_GGIR/acc.csv', index = True, header = True)

    ###### TEMP ######
    temp_data = pd.read_csv(temp_path, header = None)
    temp_start = temp_data.iloc[0,0] 
    temp = temp_data.iloc[2:].reset_index(drop=True)
    temp.index = pd.date_range(start=pd.to_datetime(temp_start, unit='s'), periods = len(temp), freq = '0.25s')
    temp.columns = ['temp']
    start_stop_nw, _ = nimbaldetach(acc['x'], acc['y'], acc['z'], temp["temp"], accel_freq=32, temperature_freq=4, quiet=True)
    start_stop_nw.to_csv(HOM_path + sub_ID + '/input_GGIR/nonwear.csv')

    if len(start_stop_nw) > 0:
        print("Non-wear detected!")
        print(start_stop_nw)
        plt.figure(figsize=(10, 6)) 
        plt.plot(acc["x"].values, label="x")
        plt.plot(acc["y"].values, label="y")
        plt.plot(acc["z"].values, label="z")
        plt.title(sub_ID, fontsize=20)
        for _, row in start_stop_nw.iterrows():
            start = row["Start Datapoint"]
            stop = row["End Datapoint"]
            plt.axvspan(start, stop, color='red', alpha=0.5)
        plt.legend(loc = "upper right")
        plt.savefig(HOM_path + sub_ID + '/output_GGIR/nonwear.png', bbox_inches='tight')

2_SL
Duration: 0 days 16:00:00


In [13]:
plt.figure()
plt.plot(acc.index, acc["x"].values)

### Sleep parameters

In [92]:
tst = {sub: [] for sub in sub_names}
waso = {sub: [] for sub in sub_names}
SE = {sub: [] for sub in sub_names}

for i, sub in enumerate(sub_names):
    output_GGIR_path = "/Volumes/Untitled/Hom3_CinC/CinC/Data_24hPulses/mod_newQuality/" + sub + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    tst[sub] = output_GGIR["SleepDurationInSpt"].values[0]

    waso[sub] = output_GGIR["WASO"].values[0]

    SE[sub] = tst[sub] / (tst[sub] + waso[sub])

In [93]:
# remove subjects with tst < 5h
tst = pd.DataFrame(tst, index = ["TST"]).T
tst_df = tst[tst["TST"] > 5]

waso = pd.DataFrame(waso, index = ["WASO"]).T
waso_df = waso[tst["TST"] > 5]

SE = pd.DataFrame(SE, index = ["SE"]).T
SE_df = SE[tst["TST"] > 5]

In [94]:
# boxplot for each subject. 3 separate subplot for each boxplot

import seaborn as sns
sns.set_context("talk")

fig, axs = plt.subplots(1, 3, figsize=(10, 6))

sns.boxplot(data = tst_df, ax = axs[0], color = sns.color_palette("Set2")[0], showfliers = False)
sns.stripplot(data = tst_df, ax = axs[0], color = "black", size = 8, alpha = 0.5)
axs[0].set_title("Total Sleep Time")
axs[0].set_ylabel("TST (hours)")

sns.boxplot(data = waso_df, ax = axs[1], color = sns.color_palette("Set2")[0], showfliers = False)
sns.stripplot(data = waso_df, ax = axs[1], color = "black", size = 8, alpha = 0.5)
axs[1].set_title("Wake After Sleep Onset")
axs[1].set_ylabel("WASO (hours)")

sns.boxplot(data = SE_df, ax = axs[2], color = sns.color_palette("Set2")[0], showfliers = False)
sns.stripplot(data = SE_df, ax = axs[2], color = "black", size = 8, alpha = 0.5)
axs[2].set_title("Sleep Efficiency")
axs[2].set_ylabel("SE (%)")

plt.tight_layout()

## Movement algorithm

In [60]:
import neurokit2 as nk

def hl_envelopes_idx(s, dmin=1, dmax=1, split=False):
    """
    Compute high and low envelopes of a signal s
    Parameters
    ----------
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases

    Returns
    -------
    lmin,lmax : high/low envelope idx of input signal s
    """

    # locals min      
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    # locals max
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
    
    if split:
        # s_mid is zero if s centered around x-axis or more generally mean of signal
        s_mid = np.mean(s) 
        # pre-sorting of locals min based on relative position with respect to s_mid 
        lmin = lmin[s[lmin]<s_mid]
        # pre-sorting of local max based on relative position with respect to s_mid 
        lmax = lmax[s[lmax]>s_mid]

    # global min of dmin-chunks of locals min 
    lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
    # global max of dmax-chunks of locals max 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
    
    return lmin,lmax

def compute_envelope(acc, resample = True):
    """
    Compute the envelope of the acceleration signal

    Parameters
    ----------
    acc : pd.Series
        Band-pass filtered accelerometer signal magnitude vector
    resample : bool, optional
        If True, resample the envelope to the original size of the signal

    Returns
    -------
    env_diff : pd.Series
        Envelope difference of the acceleration signal
    """

    lmin, lmax = hl_envelopes_idx(acc.values, dmin = 10, dmax = 10)

    # adjust shapes
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
        
    upper_envelope = acc.values[lmax]
    lower_envelope = acc.values[lmin]
                                
    if resample:
        upper_envelope_res = np.interp(np.arange(len(acc)), lmax, upper_envelope)
        lower_envelope_res = np.interp(np.arange(len(acc)), lmin, lower_envelope)
        env_diff = pd.Series(upper_envelope_res - lower_envelope_res, index = acc.index)
    else:
        env_diff = pd.Series(upper_envelope - lower_envelope, index = acc.index[lmax])

    return env_diff

def detect_bursts(acc, envelope = True, resample_envelope = True, alfa = None):
    """
    Detect bursts in acceleration signal

    Parameters
    ----------
    acc : pd.Series
        Band-pass filtered accelerometer signal magnitude vector
    envelope : bool, optional
        If True, detect bursts based on the envelope of the signal
        If False, detect bursts based on the std of the signal
    resample_envelope : bool, optional
        If True, resample the envelope to the original size of the signal
    alfa : float, optional
        Threshold for detecting bursts

    Returns
    -------
    bursts : pd.Series
        pd.DataFrame with burst start times, end times, duration, peak-to-peak amplitude, and AUC
    """

    # band-pass filter the signal
    acc = pd.Series(nk.signal_filter(acc.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = acc.index)

    if envelope:
        env_diff = compute_envelope(acc, resample = resample_envelope)
        th = alfa
    else:
        std_acc = acc.resample("1 s").std()
        std_acc.index.round("1 s")
        th = np.percentile(std_acc, 10) * alfa
        env_diff = std_acc

    bursts1 = (env_diff > th).astype(int)
    start_burst = bursts1.where(bursts1.diff()==1).dropna()
    end_burst = bursts1.where(bursts1.diff()==-1).dropna()
    if bursts1.iloc[0] == 1:
            start_burst = pd.concat([pd.Series(0, index = [bursts1.index[0]]), start_burst])
    if bursts1.iloc[-1] == 1:
        end_burst = pd.concat([end_burst, pd.Series(0, index = [bursts1.index[-1]])])
    bursts_df = pd.DataFrame({"duration": end_burst.index - start_burst.index}, index = start_burst.index)

    start = bursts_df.index
    end = pd.to_datetime((bursts_df.index + bursts_df["duration"]).values)

    end = end.to_series().reset_index(drop = True)
    start = start.to_series().reset_index(drop = True)

    duration_between_bursts = (start.iloc[1:].values - end.iloc[:-1].values)

    # If two bursts are too close to each other (5s), consider them as one burst
    for i in range(len(start)-1):
        if duration_between_bursts[i] < pd.Timedelta("5 s"):
            end[i] = np.nan
            start[i+1] = np.nan
    end.dropna(inplace = True)
    start.dropna(inplace = True)

    # extract amplitude of the bursts
    bursts = pd.DataFrame({"start": start.reset_index(drop = True), "end": end.reset_index(drop = True)})
    p2p = []
    auc = []
    for i in range(len(bursts)):
        # peak-to-peak amplitude of bp acceleration
        p2p.append(acc.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]].max() - acc.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]].min())
        # AUC of env_diff
        auc.append(np.trapz(env_diff.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]]))
    bursts["duration"] = bursts["end"] - bursts["start"]
    bursts["peak-to-peak"] = p2p
    bursts["AUC"] = auc
    return bursts

In [55]:
def compute_acc_norm(acc):
    # acc_norm = np.sqrt(np.sum(acc**2, axis=1))
    acc_norm = np.linalg.norm(acc, axis=1)
    return acc_norm

In [61]:
sns.set_context("notebook")

In [62]:
TH_WRIST = 20

# bursts_lw = detect_bursts(lw_df, resample_envelope = True, alfa = TH_WRIST)

acc = pd.read_csv(HOM_path + "/10_AC/input_GGIR/acc.csv", index_col = 0)

acc_norm  = compute_acc_norm(acc)
acc_norm = pd.Series(acc_norm, index = pd.to_datetime(acc.index))
acc_norm.head()

plt.figure()
plt.plot(acc_norm)

In [82]:
n_bursts = {sub: [] for sub in sub_names}
ampl_bursts = {sub: [] for sub in sub_names}
dur_bursts = {sub: [] for sub in sub_names}

TH_WRIST = 20

for i, sub in enumerate(sub_names):

    acc = pd.read_csv(HOM_path + "/" + sub + "/input_GGIR/acc.csv", index_col = 0)

    acc_norm  = compute_acc_norm(acc)
    acc_norm = pd.Series(acc_norm, index = pd.to_datetime(acc.index))

    # segment ACC norm during sleep 
    output_GGIR_path = "/Volumes/Untitled/Hom3_CinC/CinC/Data_24hPulses/mod_newQuality/" + sub + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(acc_norm.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(acc_norm.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(acc_norm.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])

    acc_norm_night = acc_norm.loc[sleep_onset:wake_onset] * 1000

    # detect bursts
    bursts = detect_bursts(acc_norm_night, resample_envelope = True, alfa = TH_WRIST)

    n_bursts[sub].append(len(bursts))
    ampl_bursts[sub].append(bursts["AUC"].sum() / 1000)
    dur_bursts[sub].append(bursts["duration"].sum().total_seconds()/60)

In [85]:
n_bursts_df = pd.DataFrame(n_bursts, index = ["n_bursts"]).T
ampl_bursts_df = pd.DataFrame(ampl_bursts, index = ["ampl_bursts"]).T
dur_bursts_df = pd.DataFrame(dur_bursts, index = ["dur_bursts"]).T

n_bursts_df.shape, ampl_bursts_df.shape, dur_bursts_df.shape

((39, 1), (39, 1), (39, 1))

In [87]:
n_bursts_df.to_csv("/Volumes/Untitled/Datasets_Serena/n_bursts_HOM.csv")
ampl_bursts_df.to_csv("/Volumes/Untitled/Datasets_Serena/ampl_bursts_HOM.csv")
dur_bursts_df.to_csv("/Volumes/Untitled/Datasets_Serena/dur_bursts_HOM.csv")

In [96]:
n_bursts_df = pd.read_csv("/Volumes/Untitled/Datasets_Serena/n_bursts_HOM.csv", index_col = 0)
ampl_bursts_df = pd.read_csv("/Volumes/Untitled/Datasets_Serena/ampl_bursts_HOM.csv", index_col = 0)
dur_bursts_df = pd.read_csv("/Volumes/Untitled/Datasets_Serena/dur_bursts_HOM.csv", index_col = 0)

,n_bursts
10_AC,94
11_EC,63
12_FN,160
13_MS,42
14_AD,69


In [97]:
# boxplot for each subject. 3 separate subplot for each boxplot

# remove subjects with tst < 5h
n_bursts_df = n_bursts_df[tst["TST"] > 5]
ampl_bursts_df = ampl_bursts_df[tst["TST"] > 5]
dur_bursts_df = dur_bursts_df[tst["TST"] > 5]

import seaborn as sns
sns.set_context("talk")

fig, axs = plt.subplots(1, 3, figsize=(10, 6))

sns.boxplot(data = n_bursts_df, ax = axs[0], color = sns.color_palette("Set2")[1], showfliers = False)
sns.stripplot(data = n_bursts_df, ax = axs[0], color = "black", size = 8, alpha = 0.5)
axs[0].set_title("Number of ACC Bursts")
axs[0].set_ylabel("N Bursts")

sns.boxplot(data = ampl_bursts_df, ax = axs[1], color = sns.color_palette("Set2")[1], showfliers = False)
sns.stripplot(data = ampl_bursts_df, ax = axs[1], color = "black", size = 8, alpha = 0.5)
axs[1].set_title("Total Amplitude of Bursts")
axs[1].set_ylabel("AUC (g)")

sns.boxplot(data = dur_bursts_df, ax = axs[2], color = sns.color_palette("Set2")[1], showfliers = False)
sns.stripplot(data = dur_bursts_df, ax = axs[2], color = "black", size = 8, alpha = 0.5)
axs[2].set_title("Total Duration of Bursts")
axs[2].set_ylabel("Duration (min)")

plt.tight_layout()

### Mean HR day vs night

### Load all E4 data (ACC, PPG, TEMP, EDA)

In [15]:
fs_temp = 4
fs_EDA = 4
fs_acc = 32
fs_ppg = 64
for i, sub_ID in enumerate(sub_names):
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    temp_path = glob.glob(HOM_path + sub_ID + '/**/TEMP.csv', recursive=True)[0]

    ###### ACC ######
    acc_data = pd.read_csv(acc_path, header = None)
    acc_start = acc_data.iloc[0,0] # start time in unix
    acc = acc_data.drop([0,1]).reset_index(drop=True) / 64 # convert to g
    acc.index = pd.date_range(start=pd.to_datetime(acc_start, unit='s'), periods = len(acc), freq = '0.03125s') # timestamps as index
    acc.columns = ['x', 'y', 'z']

    ###### PPG ######
    ppg_data = pd.read_csv(ppg_path, header = None)
    ppg_start = ppg_data.iloc[0,0]
    ppg = ppg_data.iloc[2:].reset_index(drop=True)
    ppg.index = pd.date_range(start=pd.to_datetime(ppg_start, unit='s'), periods = len(ppg), freq = '0.015625s')
    ppg.columns = ['ppg']

    ###### EDA ######
    eda_path = glob.glob(HOM_path + sub_ID + '/**/EDA.csv', recursive=True)[0]
    eda_data = pd.read_csv(eda_path, header = None)
    eda_start = eda_data.iloc[0,0]
    eda = eda_data.iloc[2:].reset_index(drop=True)
    eda.index = pd.date_range(start=pd.to_datetime(eda_start, unit='s'), periods = len(eda), freq = '0.25s')

    ###### TEMP ######
    temp_data = pd.read_csv(temp_path, header = None)
    temp_start = temp_data.iloc[0,0] 
    temp = temp_data.iloc[2:].reset_index(drop=True)
    temp.index = pd.date_range(start=pd.to_datetime(temp_start, unit='s'), periods = len(temp), freq = '0.25s')
    temp.columns = ['temp']

    if sub_ID == "2_SL":
        acc = acc.loc[acc.index[-1] - pd.Timedelta(hours = 16):]
        ppg = ppg.loc[ppg.index[-1] - pd.Timedelta(hours = 16):]
        eda = eda.loc[eda.index[-1] - pd.Timedelta(hours = 16):]
        temp = temp.loc[temp.index[-1] - pd.Timedelta(hours = 16):]

    plt.figure(figsize=(10, 6))
    # plt.plot(acc)
    plt.figure(figsize=(10, 6))
    plt.plot(ppg)
    plt.figure(figsize=(10, 6))
    plt.plot(eda)

32_CD
35_AA


## LookOfLife

In [ ]:
LoL_path = "/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Serena_Marcello/SleepAnalysis/Datasets/LookOfLife/"
sub_names = sorted([f for f in os.listdir(LoL_path) if not f.startswith('.')])
len(sub_names)

41

In [ ]:
# Create an input_GGIR and an output_GGIR folder for each subject
for i, sub_ID in enumerate(sub_names): 
    days = [d for d in os.listdir(LoL_path + sub_ID) if not d.startswith('.')]
    for day in days:
        input_GGIR = LoL_path + sub_ID + "/" + day + "/input_GGIR/"
        output_GGIR = LoL_path + sub_ID + "/" + day + "/output_GGIR/"
        if not os.path.exists(input_GGIR):
            os.makedirs(input_GGIR)
        if not os.path.exists(output_GGIR):
            os.makedirs(output_GGIR)
        print(f"{i+1}/{len(sub_names)} - {sub_ID} done!")

### Save acc data for GGIR and detect non-wear

In [ ]:
fs_acc = 32
for i, sub_ID in enumerate(sub_names[38:]):

    print(sub_ID)
    days = [d for d in os.listdir(LoL_path + sub_ID) if not d.startswith('.')]
    
    for day in days:

        print(day)
        acc_path = glob.glob(LoL_path + sub_ID + '/' + day + '/ACC.csv', recursive=True)[0]
        temp_path = glob.glob(LoL_path + sub_ID + '/' + day + '/TEMP.csv', recursive=True)[0]

        acc_data = pd.read_csv(acc_path, header = None)
        acc_start = acc_data.iloc[0,0] # start time in unix
        acc = acc_data.drop([0,1]).reset_index(drop=True) / 64 # convert to g
        acc.index = pd.date_range(start=pd.to_datetime(acc_start, unit='s'), periods = len(acc), freq = '0.03125S') # timestamps as index
        acc.columns = ['x', 'y', 'z']
        # print recording duration
        print(f"Duration: {acc.index[-1] - acc.index[0]}")
        # Save for GGIR
        acc.to_csv(LoL_path + sub_ID + '/' + day + '/input_GGIR/acc.csv', index = True, header = True)

        ###### TEMP ######
        temp_data = pd.read_csv(temp_path, header = None)
        temp_start = temp_data.iloc[0,0] 
        temp = temp_data.iloc[2:].reset_index(drop=True)
        temp.index = pd.date_range(start=pd.to_datetime(temp_start, unit='s'), periods = len(temp), freq = '0.25S')
        temp.columns = ['temp']

        start_stop_nw, _ = nimbaldetach(acc['x'], acc['y'], acc['z'], temp["temp"], accel_freq=32, temperature_freq=4, quiet=True)
        start_stop_nw.to_csv(LoL_path + sub_ID + '/' + day + '/input_GGIR/nonwear.csv')

        if len(start_stop_nw) > 0:
            print("Non-wear detected!")
            print(start_stop_nw)
            plt.figure(figsize=(10, 6)) 
            plt.plot(acc["x"].values, label="x")
            plt.plot(acc["y"].values, label="y")
            plt.plot(acc["z"].values, label="z")
            plt.title(sub_ID, fontsize=20)
            for _, row in start_stop_nw.iterrows():
                start = row["Start Datapoint"]
                stop = row["End Datapoint"]
                plt.axvspan(start, stop, color='red', alpha=0.5)
            plt.legend(loc = "upper right")
            plt.savefig(LoL_path + sub_ID + '/' + day + '/input_GGIR/nonwear.png', bbox_inches='tight')

## PAINLESS

In [3]:
PAIN_path = "/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Serena_Marcello/SleepAnalysis/Datasets/PAINLESS/"
PAIN_path = "/Volumes/Untitled/PAINLESS/"
sub_names = sorted([f for f in os.listdir(PAIN_path) if not f.startswith('.')])
len(sub_names)

18

In [ ]:
# Create an input_GGIR and an output_GGIR folder for each subject
days = ["Day1", "Day2"]
for i, sub_ID in enumerate(sub_names): 
    for day in days:
        input_GGIR = PAIN_path + sub_ID + "/" + day + "/input_GGIR/"
        output_GGIR = PAIN_path + sub_ID + "/" + day + "/output_GGIR/"
        if not os.path.exists(input_GGIR):
            os.makedirs(input_GGIR)
        if not os.path.exists(output_GGIR):
            os.makedirs(output_GGIR)
        print(f"{i+1}/{len(sub_names)} - {sub_ID} done!")

### Save acc data for GGIR and detect non-wear

In [7]:
fs_acc = 32

days = ["Day1", "Day2"]

for i, sub_ID in enumerate(sub_names[4:]):

    print(sub_ID)
    
    for day in days:

        print(day)
        acc_path = glob.glob(PAIN_path + sub_ID + '/' + day + '/ACC.csv', recursive=True)[0]
        temp_path = glob.glob(PAIN_path + sub_ID + '/' + day + '/TEMP.csv', recursive=True)[0]

        acc_data = pd.read_csv(acc_path, header = None)
        acc_start = acc_data.iloc[0,0] # start time in unix
        acc = acc_data.drop([0,1]).reset_index(drop=True) / 64 # convert to g
        acc.index = pd.date_range(start=pd.to_datetime(acc_start, unit='s'), periods = len(acc), freq = '0.03125S') # timestamps as index
        acc.columns = ['x', 'y', 'z']
        # print recording duration
        print(f"Duration: {acc.index[-1] - acc.index[0]}")
        # Save for GGIR
        acc.to_csv(PAIN_path + sub_ID + '/' + day + '/input_GGIR/acc.csv', index = True, header = True)

        ###### TEMP ######
        temp_data = pd.read_csv(temp_path, header = None)
        temp_start = temp_data.iloc[0,0] 
        temp = temp_data.iloc[2:].reset_index(drop=True)
        temp.index = pd.date_range(start=pd.to_datetime(temp_start, unit='s'), periods = len(temp), freq = '0.25S')
        temp.columns = ['temp']

        start_stop_nw, _ = nimbaldetach(acc['x'], acc['y'], acc['z'], temp["temp"], accel_freq=32, temperature_freq=4, quiet=True)
        start_stop_nw.to_csv(PAIN_path + sub_ID + '/' + day + '/input_GGIR/nonwear.csv')

        if len(start_stop_nw) > 0:
            print("Non-wear detected!")
            print(start_stop_nw)
            plt.figure(figsize=(10, 6)) 
            plt.plot(acc["x"].values, label="x")
            plt.plot(acc["y"].values, label="y")
            plt.plot(acc["z"].values, label="z")
            plt.title(sub_ID, fontsize=20)
            for _, row in start_stop_nw.iterrows():
                start = row["Start Datapoint"]
                stop = row["End Datapoint"]
                plt.axvspan(start, stop, color='red', alpha=0.5)
            plt.legend(loc = "upper right")
            plt.savefig(PAIN_path + sub_ID + '/' + day + '/input_GGIR/nonwear.png', bbox_inches='tight')

005
Day1
Duration: 0 days 23:56:39.156250
Day2
Duration: 0 days 23:30:11.031250
Non-wear detected!
   Start Datapoint  End Datapoint
1           967184         983256
2          1128904        1164136
006
Day1
Duration: 0 days 23:01:55.468750
Day2
Duration: 0 days 23:59:27.343750
007
Day1
Duration: 0 days 23:55:30.531250
Non-wear detected!
   Start Datapoint  End Datapoint
1          2460872        2509072
Day2
Duration: 0 days 23:48:29.406250
Non-wear detected!
   Start Datapoint  End Datapoint
1          2312496        2336704
008
Day1
Duration: 0 days 22:36:48.906250
Non-wear detected!
   Start Datapoint  End Datapoint
1          1088320        1099408
Day2
Duration: 1 days 00:04:16.656250
Non-wear detected!
   Start Datapoint  End Datapoint
1          1051656        1071472
009
Day1
Duration: 0 days 23:15:33.156250
Day2
Duration: 1 days 00:16:05.218750
Non-wear detected!
   Start Datapoint  End Datapoint
1          2664856        2694256
010
Day1
Duration: 0 days 23:02:44.218750
Da